In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Heart_rate/GSE236927'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Inflammatory markers and frailty in home-dwelling elderly, a cross-sectional study"
!Series_summary	"Low-grade, chronic inflammation during ageing  (“inflammageing”) is suggested to be involved in the development of frailty in older age. However, studies on the association between frailty, using the frailty index definition, and inflammatory markers are limited."
!Series_summary	"The aim of this study was to investigate the relationship between inflammatory markers and frailty index (FI) in older, home-dwelling adults. Home-dwelling men and women aged ≥ 70 years old, living in South-East Norway were recruited and included in a cross-sectional study. The FI used in the current study was developed according to Rockwood’s frailty index and included 38 variables, resulting in an FI score between 0 and 1 for each participant. Circulating inflammatory markers (IL-6, CRP, IGF-1, cystatin C, cathepsin S, and glycoprotein Acetyls) were analyzed from non-fa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True

# Variable Availability
# Heart_rate
trait_row = None  # No heart rate data found

# Age
for key, vals in clinical_data.items():
    if 'age (y)' in vals[0]:
        age_row = key
        break

# Gender
for key, vals in clinical_data.items():
    if 'sex (female/male)' in vals[0]:
        gender_row = key
        break
    
# Data Type Conversion Functions
def convert_age(value):
    try:
        val = float(value.split(":")[1].strip())
        return val
    except:
        return None

def convert_gender(value):
    gender_map = {'female': 0, 'male': 1}
    try:
        val = value.split(":")[1].strip().lower()
        return gender_map.get(val, None)
    except:
        return None

# Save Metadata
save_cohort_info('GSE236927', './preprocessed/Heart_rate/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction - skip if trait_row is None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Heart_rate', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Heart_rate/trait_data/GSE236927.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
